In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import math

from collections import Counter
from sklearn import preprocessing

from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, confusion_matrix

import random
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf

import random

import gensim
import gensim.downloader
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_file = pd.read_csv('clean.csv')

In [ ]:
train_file.columns, train_file.shape

In [ ]:
max_label_length = 6
new_label = []
for each in train_file['label']:
    temp = str(each)
    if len(temp) != max_label_length:
        zeros = max_label_length - len(str(each))
        res = temp.rjust(zeros + len(temp), '0')
        new_label.append(res)
    else:
        new_label.append(temp)    

In [ ]:
train_file['label'] = new_label

In [ ]:
train_file = train_file.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis = 1)


In [ ]:
value_counts_label = train_file['label'].value_counts()
value_counts_label

In [ ]:
labels_one_dp = ['110110', '110101']
inds = train_file[ (train_file['label'] == '110110') ].index
train_file.drop(inds , inplace=True)
train_file.shape

inds = train_file[ (train_file['label'] == '110101') ].index
train_file.drop(inds , inplace=True)
train_file.shape

In [ ]:
train, val = train_test_split(train_file, test_size=0.1, random_state=42, stratify = train_file['label'], shuffle = True)


In [ ]:


x_train = train['comment_text'].reset_index(drop = True)
y_train = train['label'].reset_index(drop = True)

x_val = val['comment_text'].reset_index(drop = True)
y_val = val['label'].reset_index(drop = True)


del train
del val


#### Split the comment and get the vectors for words
#### if the word is OOV, simply skip, and in process if list is empty then append 0
#### if sentence vectors by average: simple take mean
#### else return all vectors

In [ ]:
def sentence_vectors(datapoint, vector_type):
    words = datapoint.split(' ')
    word_vectors = []
    for each in words:
        try:
            word_vectors.append(fasttext_vectors[each])
        except KeyError:
            pass
    word_vectors = np.array(word_vectors)
    
    if vector_type == 'sent':
        if word_vectors.shape[0] == []:
            return 0
        else:
            return np.mean(word_vectors, axis = 0)
    else:
        if word_vectors.shape[0] == []:
            return 0, 0
        else:
            return word_vectors, len(words)

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
fasttext_vectors = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
train_word_vector = x_train.apply(sentence_vectors, vector_type = 'word')
val_word_vector = x_val.apply(sentence_vectors, vector_type = 'word')


In [ ]:
def get_vts_and_lgts(train_word_vector):
    lengths = []
    vectors = []
    for each in range(train_word_vector.shape[0]):
        lengths.append(train_word_vector.iloc[each][1])
        vectors.append(train_word_vector.iloc[each][0])
    return vectors, lengths


In [ ]:
vectors_train, lengths_train = get_vts_and_lgts(train_word_vector)
vectors_val, lengths_val = get_vts_and_lgts(val_word_vector)

del train_word_vector, val_word_vector


In [ ]:
max_len = max(max(lengths_train), max(lengths_val))


#### Get indices where 0 is returned and discard them

In [ ]:
def get_index_zerovectors(train_sentence_vector):
    remove_indices = []
    for index, vector in enumerate(train_sentence_vector):
        if vector.shape == ():
            remove_indices.append(index)
    return remove_indices

In [ ]:
train_remove_indices = get_index_zerovectors(vectors_train)
val_remove_indices = get_index_zerovectors(vectors_val)


In [ ]:
vectors_train = [i for j, i in enumerate(vectors_train) if j not in train_remove_indices]
y_train = [i for j, i in enumerate(y_train) if j not in train_remove_indices]

vectors_val = [i for j, i in enumerate(vectors_val) if j not in val_remove_indices]
y_val = [i for j, i in enumerate(y_val) if j not in val_remove_indices]



#### Make lengths of all comments same as max_len with appending 0.0 at the start

In [ ]:
vectors_train = tf.keras.utils.pad_sequences(
    vectors_train,
    maxlen=max_len,
    dtype='int32',
    padding='pre',
    truncating='pre',
    value=0.0
)
vectors_val = tf.keras.utils.pad_sequences(
    vectors_val,
    maxlen=max_len,
    dtype='int32',
    padding='pre',
    truncating='pre',
    value=0.0
)



In [ ]:
def string_to_array(y):
    Y_ = []
    for each in y:
        a = []
        temp = list(each)
        for every in temp:
            a.append(int(every))
        np.array(a)
        Y_.append(a)    
    return np.array(Y_)

In [ ]:
Y_train = string_to_array(y_train)
Y_val = string_to_array(y_val)

In [ ]:
vectors_train.shape, Y_train.shape, vectors_val.shape, Y_val.shape


In [ ]:
def rnn():
    inputs = tf.keras.Input(shape = (1250, 300))
    x = tf.keras.layers.SimpleRNN(16, activation = 'tanh')(inputs)
    outputs = tf.keras.layers.Dense(6, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.8),
]

model_rnn = rnn()
model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
model_rnn.summary()


In [ ]:
history_rnn = model_rnn.fit(x = vectors_train, y = Y_train, validation_data=(vectors_val, Y_val), epochs=3)


#### Get predictions, and where the value is >threshold change to 1 else 0
#### We have 6 classes. In multi-label classification - classes are not mutually exclusive
#### calculate precision, recall and f1_score for individual classes
#### How our model is performing overall -> take the average all precision, recall, f1_score.

In [ ]:
preds_rnn = model_rnn.predict(vectors_val)
preds_rnn = np.where(preds_rnn > 0.7, 1, 0)

p_avg, r_avg, f1_avg = 0,0,0
for i in range(6):
    real = Y_val[0:, i]
    reel = preds_rnn[0:, i]
    conf_mat = confusion_matrix(real, reel)

    print(conf_mat)
    p = precision_score(real, reel)
    r = recall_score(real, reel)
    f1 = f1_score(real, reel)

    print('p---', p)
    print('r---', r)
    print('f1---', f1)

    p_avg += p
    r_avg += r
    f1_avg += f1

    print('************************')
print(p_avg / 6, r_avg /6, f1_avg / 6)

#### Bahdanau Attention

In [ ]:
class attention(tf.keras.layers.Layer):
    def __init__(self, inputs_0, inputs_1):
        super().__init__()
        self.W = self.add_weight(name = 'attention_weight', shape = (inputs_1, 1),
                             initializer = 'random_normal', trainable = True)
        self.b = self.add_weight(name = 'attention_bias', shape = (inputs_0, 1),
                             initializer = 'zeros', trainable = True)
    
    def call(self, inputs):
        e = tf.math.tanh(tf.matmul(inputs, self.W) + self.b)
        e = tf.squeeze(e, axis=-1, name='squeeze')
        alpha = tf.nn.softmax(e)
        alpha = tf.expand_dims(alpha, axis = -1)
        context = inputs * alpha
        context = tf.math.reduce_sum(context, axis = -1)
        return context

In [ ]:
def rnn_attention():
    inputs = tf.keras.Input(shape = (1250, 300))
    x = tf.keras.layers.SimpleRNN(8, return_sequences = True, activation = 'tanh')(inputs)
    x = attention(x.shape[1], x.shape[2])(x)
    outputs = tf.keras.layers.Dense(6, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.8),
      
]

model_attention = rnn_attention()
model_attention.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
model_attention.summary()


In [ ]:
history_rnnatten = model_attention.fit(x = vectors_train, y = y_train, validation_data=(vectors_val, y_val), epochs=3)


In [ ]:
preds_attn = model_attention.predict(vectors_val)
preds_attn = np.where(preds_attn > 0.7, 1, 0)

p_avg, r_avg, f1_avg = 0,0,0
for i in range(6):
    real = Y_val[0:, i]
    reel = preds_attn[0:, i]
    conf_mat = confusion_matrix(real, reel)

    print(conf_mat)
    p = precision_score(real, reel)
    r = recall_score(real, reel)
    f1 = f1_score(real, reel)

    print('p---', p)
    print('r---', r)
    print('f1---', f1)

    p_avg += p
    r_avg += r
    f1_avg += f1

    print('************************')
print(p_avg / 6, r_avg /6, f1_avg / 6)



#### Weighted attention

In [ ]:
class wtgd_attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attn_weights = tf.nn.softmax(self.V(score), axis = 1)
        context_vector = tf.math.reduce_sum(attn_weights * features, axis = 1)
        return context_vector, attn_weights

In [ ]:
def bilstm_attention():
    inputs = tf.keras.Input(shape = (1250, 300))
    lstm, f_h, f_c, b_h, b_c = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8, return_sequences = True, return_state=True, activation = 'tanh'))(inputs)
    state_h = tf.keras.layers.Concatenate()([f_h, b_h])
    state_c = tf.keras.layers.Concatenate()([f_c, b_c])
    vector, weights = wtgd_attention(10)(lstm, state_h)
    outputs = tf.keras.layers.Dense(6, activation = 'sigmoid')(vector)
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.8),
      
]

bilstm_attention = bilstm_attention()
bilstm_attention.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
bilstm_attention.summary()



In [ ]:
history_bilstm = bilstm_attention.fit(x = vectors_train, y = y_train, validation_data=(vectors_val, y_val), epochs=3)


In [ ]:
preds_biattn = bilstm_attention.predict(vectors_val)
preds_biattn = np.where(preds_biattn > 0.7, 1, 0)

p_avg, r_avg, f1_avg = 0,0,0
for i in range(6):
    real = Y_val[0:, i]
    reel = preds_biattn[0:, i]
    conf_mat = confusion_matrix(real, reel)

    print(conf_mat)
    p = precision_score(real, reel)
    r = recall_score(real, reel)
    f1 = f1_score(real, reel)

    print('p---', p)
    print('r---', r)
    print('f1---', f1)

    p_avg += p
    r_avg += r
    f1_avg += f1

    print('************************')
print(p_avg / 6, r_avg /6, f1_avg / 6)

